In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from selenium import webdriver

url_main = 'https://comic.naver.com/' # webtoon/finish 완결, webtoon/detail?titleId=xxx&no=1 1화
driver_path = 'data/chromedriver.exe'

# 19세 이상 웹툰 데이터 수집을 위한 selenium 사용
driver = webdriver.Chrome(executable_path=driver_path)

In [2]:
driver.get(url_main + 'webtoon/finish')

직접 로그인 진행

In [8]:
def get_webtoon_titleId(url_main=url_main):
    r = requests.get(url_main + 'webtoon/finish').text
    soup = BeautifulSoup(r, 'lxml')
    a_list = soup.select('div.thumb a')
    
    webtoon_details = []
    epi_url_list = []
    comment_url_list = []
    
    for a in a_list:
        webtoon_detail = url_main + 'webtoon/list' + a.attrs['href'].split('list')[-1]
        epi_url = url_main + 'webtoon/detail' + a.attrs['href'].split('list')[-1]
        comment_url = url_main + 'comment/comment' + a.attrs['href'].split('list')[-1]
        
        webtoon_details.append(webtoon_detail)
        epi_url_list.append(epi_url)
        comment_url_list.append(comment_url)
        
    return webtoon_details, epi_url_list, comment_url_list

In [9]:
webtoon_details, epi_url_list, comment_url_list = get_webtoon_titleId()

In [10]:
webtoon_details[:10]

['https://comic.naver.com/webtoon/list?titleId=780986',
 'https://comic.naver.com/webtoon/list?titleId=780987',
 'https://comic.naver.com/webtoon/list?titleId=780984',
 'https://comic.naver.com/webtoon/list?titleId=749456',
 'https://comic.naver.com/webtoon/list?titleId=733079',
 'https://comic.naver.com/webtoon/list?titleId=780857',
 'https://comic.naver.com/webtoon/list?titleId=780983',
 'https://comic.naver.com/webtoon/list?titleId=776541',
 'https://comic.naver.com/webtoon/list?titleId=755674',
 'https://comic.naver.com/webtoon/list?titleId=655749']

In [33]:
soup = BeautifulSoup(driver.page_source, 'lxml')
stars = soup.select('div.rating_type strong')
stars = [float(star.get_text()) for star in stars]
stars[:10]

[9.31, 9.48, 9.8, 9.98, 9.92, 9.29, 9.93, 9.87, 9.94, 9.97]

In [34]:
len(webtoon_details), len(epi_url_list), len(comment_url_list), len(stars)

(1064, 1064, 1064, 1064)

# 회차별 댓글 수집
- 2013년 5월 이후 업로드된 작품만 수집(베댓 시스템 생긴 시점)
- 좋아요(하트) 100 이상의 작품만 수집(다른 플랫폼에서 완결되어 넘어온 웹툰 제외)

In [14]:
soup = BeautifulSoup(driver.page_source, 'lxml')
soup.select('em.u_cnt')[0].get_text()

'99,999+'

In [15]:
int(soup.select('em.u_cnt')[0].get_text().replace(',', '').replace('+', ''))

99999

## 함수 생성

In [19]:
import time
import datetime

basedate = datetime.date(2013, 5, 1)

def get_comment_epi(webtoon_detail, epi_url, comment_url, star, after=basedate, no=1):
    """
    webtoon_detail : 작품의 상세 페이지 (좋아요 수를 가져오기 위한 url)
    epi_url : 회차 base url (작품 제목과 날짜를 가져오기 위한 url)
    comment_url : 회차 댓글 url (베스트 댓글 수 등을 가져오기 위한 url)
    star : 평균 평점 (예측을 위한 값(y))
    after : 베댓 시스템이 생긴 2013년 5월 이후로만 수집
    no : 회차
    """
    
    # 좋아요 수를 추출하여 100 미만이면 pass
    driver.get(webtoon_detail)
    time.sleep(0.5)
    soup = BeautifulSoup(driver.page_source, 'lxml')
    like = soup.select('em.u_cnt')
    
    # 좋아요 수가 0인 경우
    if len(like) == 0:
        return
    
    like = int(like[0].get_text().replace(',', '').replace('+', ''))
    if like < 100:
        return
    
    # 우선 날짜를 추출 하여 2013년 5월 전에 올라온 회차면 pass
    driver.get(epi_url + '&no={}'.format(no))
    time.sleep(0.5)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    date = soup.select('.date')[0].get_text()
    date = datetime.date(int(date.split('.')[0]), int(date.split('.')[1]), int(date.split('.')[2]))
    
    if date < basedate:
        return
    
    # 제목과 평점, 평가한 인원 수 가져오기
    title = soup.select('.detail')[0].get_text().split('\t')[0].strip()
    score = float(soup.select('#topPointTotalNumber')[0].get_text())
    score_num = int(soup.select('.pointTotalPerson em')[0].get_text())
    
    # 베스트 댓글 수, 내용, 추천 합, 비추천 합 추출
    driver.get(comment_url + '&no={}'.format(no))
    time.sleep(0.5)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    best_comments = soup.select('.u_cbox_contents')
    
    best_comments_num = len(best_comments)
    best_comments = [comment.get_text() for comment in best_comments]
    best_comments = '\n'.join(best_comments) # 댓글을 하나로 합침
    
    recommen = soup.select('.u_cbox_cnt_recomm')
    recommen = [int(num.get_text()) for num in recommen]
    sum_recommen = sum(recommen)
    
    unrecomm = soup.select('.u_cbox_cnt_unrecomm')
    unrecomm = [int(num.get_text()) for num in unrecomm]
    sum_unrecomm = sum(unrecomm)
    
    # 전체 댓글 수, 최근 15개 댓글 내용 추출
    comments_num = int(soup.select('.u_cbox_count')[0].get_text().replace(',', ''))
    
    driver.find_element_by_css_selector('#cbox_module_wai_u_cbox_sort_option_tab2 > span.u_cbox_sort_label').click()
    time.sleep(0.5)
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    comments_ = soup.select('.u_cbox_contents')
    comments_ = [comment.get_text() for comment in comments_]
    comments_ = '\n'.join(comments_)
    
    return [title, score, score_num, best_comments_num, best_comments, sum_recommen, sum_unrecomm, comments_num, comments_, star]

In [27]:
# 테스트 (좋아요 수 71)
result = get_comment_epi(webtoon_details[15], epi_url_list[15], comment_url_list[15], stars[15])
result

In [29]:
# 테스트2 (2012년 작품)
result = get_comment_epi('https://comic.naver.com/webtoon/list?titleId=318995', 'https://comic.naver.com/webtoon/detail?titleId=318995', 'https://comic.naver.com/comment/comment?titleId=318995', 0)
result

In [31]:
# 테스트3
result = get_comment_epi(webtoon_details[14], epi_url_list[14], comment_url_list[14], stars[14])
result

['롤랑롤랑',
 9.96,
 46931,
 15,
 '완전 동화책 같다...\n강아지들 빵실빵실할게 너무 귀엽다ㅠㅠㅠ\n아 자유님!  정말 기다렸어요얼마만에 나온 신작인지 다른작품도재밌는데비인기라는게 넘 아쉬웠습니다 점수매기자면백점만점중 백점인데ㅠㅠ괄호열고 전에있던 작품도 재연재하였으면... 좋겠네요이 작가님은 다른작품들도 재밌어서 쿠키 구워드리고 싶은데...  괄호닫고만화 재밌게 잘보겠습니다세계에서 제일가는 작가가 됬음 좋겠어요^^\n몰랑 몰랑한 개 볼 만지고 싶다 몰랑 몰랑\n헉 너무 아기자기해ㅠㅠㅠㅠ완전 기대된다\n웰시코기하면 꼭 저 이야기 하시는 분들이 계시는데.. 있는 웰시코기가 있고 없는 웰시코기가 따로 있습니다 무조건 학대당한 동물이라고 어필하진 말아주세요...\n저희집 웰시코기는 꼬리가 원래 짧은 팸브룩 종입니다. 모든 웰시코기가 다 자르지는 않아요. 가디건 종도 단미하지 않는게 추세로 떠오르고 있구요. 그리고 부득이한 미용목적의 단미도 병원에서 충분히 안전하게 수술을 통해서 단미합니다. 웰시코기 견주로써 정확하지 않은 의견이 다수인양 퍼지는게 슬퍼서요.\n별점 테러 진짜 그만 좀 하자\n베댓분이 하신 말씀이 틀린 건 아니지만 태어날 때 부터 꼬리 없는 웰시코기는 없습니다.자연 단미인 아이들도 꼬리가 짧을 뿐 형태는 있어요. (짧고 뭉툭함)한국에서 보이는 대부분의 펨브룩도 꼬리의 형태가 없다면 단미수술을 거친 아이들입니다. 자연 단미는 거의 없다고 봐야해요.꼬리가 아예 없는 것 처럼 보이는 웰시코기들은 전부 단미 수술한 애들이 맞아요..\n베너사진 왹굳형이랑 메시 인줄\n아 인챈트 작가님이시구나 그림체가 전작과 달라보여서 헷갈렸어요ㅎㅎ 이번 작품도 잘 부탁드립니다!\n다들 웰시코기 꼬리 얘기만 하시는데 저는 그것보다 다른 동물들은 모두 인간을 없애려? 하는데 개들만이 인간을 용서해달라고 빌었던 장면에서 정말 울컥했네요. 지금도 인간에 의해 고통받는 개들이 많을텐데 진짜 개들에게 미안하고 또 인간과 같이 살아줘서 고마워요\n나도 어쩔 수 없는 침

In [35]:
# 코드 돌리기 전에 반드시 클린봇 OFF로!!

import pandas as pd
from tqdm import tqdm

results = []
for webtoon_detail, epi_url, comment_url, star in tqdm(zip(webtoon_details, epi_url_list, comment_url_list, stars), total=len(epi_url_list)):
    result = get_comment_epi(webtoon_detail, epi_url, comment_url, star)
    if result is None:
        continue
    results.append(result)
    
df = pd.DataFrame(results, columns=['title', 'score', 'score_num', 'best_num', 'best', 'best_recomm', 'best_unrecomm', 'comment_num', 'comment', 'star'])
df.head()

100%|██████████████████████████████████████████████████████████████████████████████| 1064/1064 [44:44<00:00,  2.52s/it]


,title,score,score_num,best_num,best,best_recomm,best_unrecomm,comment_num,comment,star
0,웰캄투실버라이프,9.94,19878,15,아 할머니 귀여우시다 힐링툰 감이 오네요 잘볼게용!\n시리얼에 뜨거운 우유...나만...,52588,599,2220,정주행 시작\n솔찍히 자는데 깨우면 짜증남..할머니라도..맘은 이해하지만 더 자고 ...,9.98
1,원수를 사랑하라,9.95,31528,15,근데 여주 대단하다...엄마랑 아빠가 저런 사람들인데 멀끔히 차려입고 면접보는 거면...,129562,3792,987,근데 나 자신을 믿는것.. 지금 내가 제일 필요한 말임..ㅠㅠㅠㅠㅠ 어제 면접 말려...,9.92
2,약초마을 연쇄살초사건,9.73,10367,15,아니ㅋㅋㅋㅋㅋㅋ작가님ㅋㅋㅋㅋㅋㅋ이제 작물 쪽으로 길을 트신건가요ㅋㅋㅋㅋㅋㅋㅋㅋ\n팀...,63294,255,1462,이거 왤케 재밌냨ㅋ 팀장님 죽었는데 뒷담 까는거 보소\n약초마을 연쇄살인도 아닌 연...,9.87
3,AI가 세상을 지배한다면,9.88,12207,15,그대가 POGO 싶었소\n라움 코드 자기가 넣었다고... 이리 당당하게 선전POGO...,59238,967,2036,완결보고 정주행만 3번째 볼때마다 감정이 다름 영화로 나오기엔 잘라내거나 스토리에 ...,9.94
4,닥터 프로스트 시즌 3~4,9.95,30579,15,아무생각없이 내렸다가 이거 보고 놀란사람 손!!!\n헐?\n제가 이 웹툰을 보면서 ...,119758,2817,9707,퍄ㅑㅑ\n닥프 1기 2기 무료로 풀려있으니까 꼭보세요\n아 이런 띵작은 유료되기전에...,9.97


In [36]:
df.to_csv('data/naver_comment_2.csv', encoding='utf-8-sig', index=False)

In [37]:
driver.close()